In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader

### 3.1 数据集加载+预处理

In [2]:
batch_size=256

In [3]:
train_set = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),# 将数据转为Tensor类型,数据格式torch(C,H,W)，像素值范围【0.0,1.0】
        transforms.Normalize([0.5], [0.5])])#均值0.5方差0.5
)
train_loader=DataLoader(train_set,shuffle=True, batch_size=batch_size)

In [4]:
test_set = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),# 将数据转为Tensor类型,数据格式torch(C,H,W)，像素值范围【0.0,1.0】
        transforms.Normalize([0.5], [0.5])])#均值0.5方差0.5
)
test_loader=DataLoader(test_set,shuffle=True, batch_size=batch_size)

In [5]:
print("train_data:", train_set.train_data.size())
print("train_labels:", train_set.train_labels.size())
print("test_data:", test_set.test_data.size())

train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])


D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


#### 网络构建

In [6]:
class Model(nn.Module):
    def __init__(self,inputs,outputs):
        super(Model,self).__init__()
        self.hidden1=nn.Linear(784,512)
        self.hidden2=nn.Linear(512,512)
        self.output=nn.Linear(512,10)
    
    def forward(self,x):
        x=F.relu(self.hidden1(x))
        x=F.relu(self.hidden2(x))
        out=self.output(x)
        return out
    
model=Model(784,10)

### 3.3 损失函数和优化器

In [7]:
criterion = nn.CrossEntropyLoss()#交叉熵损失
optimizer = optim.AdamW(model.parameters())

### 3.4 训练模型

In [8]:
EPOCHS=20
for epoch in range(EPOCHS):
    train_loss=0
    num_correct=0
    train_acc=0
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    for data in train_loader:
        img, label = data# 数据送入定义好的模型
        
        img=torch.flatten(img,start_dim=1) #展平数据，转化为[256,784]
        img = torch.autograd.Variable(img)
        label = torch.autograd.Variable(label)
        
        out = model(img)
        optimizer.zero_grad()
        loss = criterion(out, label)
        loss.backward()# 反向传播
        optimizer.step()
        
        train_loss += loss.data
        _,pred = torch.max(out.data,1)#第1维度上的最大值，pred为标签序号
        num_correct+=torch.sum(pred==label.data)
    print(' Train Loss: {:.6f}, Train Acc: {:.6f}%'.format(train_loss/len(train_loader), 100*num_correct/len(train_set)))

Epoch 1/20
 Train Loss: 0.521423, Train Acc: 80.921669%
Epoch 2/20
 Train Loss: 0.365273, Train Acc: 86.574997%
Epoch 3/20
 Train Loss: 0.327439, Train Acc: 88.019997%
Epoch 4/20
 Train Loss: 0.302599, Train Acc: 88.901665%
Epoch 5/20
 Train Loss: 0.278364, Train Acc: 89.716667%
Epoch 6/20
 Train Loss: 0.262175, Train Acc: 90.186668%
Epoch 7/20
 Train Loss: 0.249076, Train Acc: 90.628334%
Epoch 8/20
 Train Loss: 0.233122, Train Acc: 91.224998%
Epoch 9/20
 Train Loss: 0.226596, Train Acc: 91.525002%
Epoch 10/20
 Train Loss: 0.210697, Train Acc: 92.025002%
Epoch 11/20
 Train Loss: 0.201143, Train Acc: 92.513336%
Epoch 12/20
 Train Loss: 0.190069, Train Acc: 92.800003%
Epoch 13/20
 Train Loss: 0.181037, Train Acc: 93.046669%
Epoch 14/20
 Train Loss: 0.172715, Train Acc: 93.449997%
Epoch 15/20
 Train Loss: 0.163770, Train Acc: 93.706665%
Epoch 16/20
 Train Loss: 0.155098, Train Acc: 94.193336%
Epoch 17/20
 Train Loss: 0.146433, Train Acc: 94.551666%
Epoch 18/20
 Train Loss: 0.137705, Train

#### 模型评估

In [10]:
model.eval()
for epoch in range(EPOCHS):
    test_loss=0#初始化
    test_acc=0
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    for i, (datas, labels) in enumerate(test_loader):
        img, label = datas,labels# 数据送入定义好的模型
        img = img.view(img.size(0), -1) # 改变数据的size，变成784*1
        img = torch.autograd.Variable(img)
        label = torch.autograd.Variable(label)
        out = model(img)
        loss = criterion(out, label)
        test_loss += loss.data
        _,pred = out.max(1)
        test_acc+=torch.sum(pred==label.data)
    print('Test Loss: {:.6f},Test Acc: {:.6f}%'.format( test_loss/len(test_loader),
                                                       100*test_acc/len(test_set)))

Epoch 1/20
Test Loss: 0.381978,Test Acc: 89.260002%
Epoch 2/20
Test Loss: 0.382553,Test Acc: 89.260002%
Epoch 3/20
Test Loss: 0.376652,Test Acc: 89.260002%
Epoch 4/20
Test Loss: 0.369521,Test Acc: 89.260002%
Epoch 5/20
Test Loss: 0.384391,Test Acc: 89.260002%
Epoch 6/20
Test Loss: 0.383326,Test Acc: 89.260002%
Epoch 7/20
Test Loss: 0.397584,Test Acc: 89.260002%
Epoch 8/20
Test Loss: 0.384324,Test Acc: 89.260002%
Epoch 9/20
Test Loss: 0.371243,Test Acc: 89.260002%
Epoch 10/20
Test Loss: 0.377408,Test Acc: 89.260002%
Epoch 11/20
Test Loss: 0.369015,Test Acc: 89.260002%
Epoch 12/20
Test Loss: 0.376843,Test Acc: 89.260002%
Epoch 13/20
Test Loss: 0.379978,Test Acc: 89.260002%
Epoch 14/20
Test Loss: 0.384850,Test Acc: 89.260002%
Epoch 15/20
Test Loss: 0.376936,Test Acc: 89.260002%
Epoch 16/20
Test Loss: 0.372023,Test Acc: 89.260002%
Epoch 17/20
Test Loss: 0.369902,Test Acc: 89.260002%
Epoch 18/20
Test Loss: 0.376379,Test Acc: 89.260002%
Epoch 19/20
Test Loss: 0.387558,Test Acc: 89.260002%
Ep

对变量进行变换时要留意之前是否对其有定义，要对其进行初始化后再运算

进行矩阵的运算时矩阵的size要对上，可中断设置输出点检查